# Mean curvature flow
In this tutorial we simulate mean curvature flows. Let $S_0$ be a closed surface with outer normal vector $\nu$ and mean curvature $H=\mathrm{tr}(\nabla_S\nu)$. Each point $X$ on the surface moves with velocity $H\,\nu$, i.e. we have the equation
$$
\partial_t X = -H\,\nu
$$
or in weak form by using that $H\,\nu=-\Delta_S X$ with the Laplace-Beltrami operator $\Delta_S$
$$
\int_{S_t}\partial_t X\cdot v\,ds = -\int_{S_t}\nabla_{S_t}X:\nabla_{S_t} v\,ds \qquad \forall v\in H^1(S_t,\mathbb{R^3}).
$$
Using the mass and stiffness forms
$$
m_t(X,v)=\int_{S_t}X\cdot v\,ds,\qquad a_t(X,v):=\int_{S_t}\nabla_{S_t}X:\nabla_{S_t} v\,ds
$$
the equation reads
$$
m_t(\partial_tX,v)+a_t(X,v)=0.
$$

We use Dziuk's method for time discretization
$$
m_{t^n}(\frac{X^{n+1}-X^n}{\tau},v)+a_{t^n}(X^{n+1},v)=0,
$$
which reads in operator form
$$
(M_n+\tau A_n)(X^{n+1}-X^n) = -\tau\,A_nX^{n}.
$$
Thus, the update scheme is given by
$$
X^{n+1} = X^n - \tau (M_n+\tau A_n)^{-1}A_nX^{n}.
$$

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

Define polynomial approximation order and timestep. We start with a simple sphere with radius $R$.

In [ ]:
order = 2
tau = 2e-4

R = 1

sphere = Sphere((0,0,0),R).faces[0]
mesh = Mesh(OCCGeometry(sphere).GenerateMesh(maxh=0.15)).Curve(order)
Draw(mesh)

Use vector-valued Lagrangian finite elements for discretizing the position field $X$ and the displacement $u=X^n-X^0$.

In [ ]:
fesH = VectorH1(mesh,order=order)
u,v = fesH.TnT()

# current displacement
gfset = GridFunction(fesH)

# position of points
gfX = GridFunction(fesH)
# old position of points
gfXold = GridFunction(fesH)

Define the bilinear form $M_n+\tau A_n$, which has to be inverted and the stiffness matrix $A_n$. Note, that the forms have to be assembled on the surface at time $t^n$. Therefore we use the option ``deformation=gfset`` to tell the integrator to perform a change of variables and use the chain-rule for computing the appropriate integrals.

In [ ]:
Mstar = BilinearForm(fesH)
Mstar += (u*v + tau*InnerProduct(Grad(u).Trace(),Grad(v).Trace()))*ds(deformation=gfset)
Mstar.Assemble()

# precompute initial inverse
inv = Mstar.mat.Inverse(fesH.FreeDofs(), inverse="sparsecholesky")

A = BilinearForm(fesH)
A += InnerProduct(Grad(u).Trace(),Grad(v).Trace())*ds(deformation=gfset)
A.Assemble();

The sphere will shrink to a point in finite time. The exact solution has radius
$$
R(t) = \sqrt{R_0^2-4\,t}
$$
and at time $T_{\mathrm{max}}=\frac{R_0^2}{4}$ the radius becomes zero.

Initialize $X^0=\mathrm{id}$ and zero displacement.

In [ ]:
t = 0

# time at which point is reached = R**2/4
Tend = R**2/4-1e-2
ex_R = sqrt(R**2-4*t)

# Reset initial data
# displ = 0,  X = id
gfset.vec[:] = 0
gfX.Set( CF( (x,y,z) ), definedon=mesh.Boundaries(".*"))

# auxiliary vector for time stepping scheme
r = gfset.vec.CreateVector()

scene = Draw(Norm(gfset), mesh, deformation=gfset)

During the time-stepping we save the old position, assemble the matrices at the current surface and update the positions. Then the displacement gets updated.

In [ ]:
i = 0
t = 0
with TaskManager():
    while t < Tend:
        print ("\rt=", t, end="")
        
        # save old position
        gfXold.vec.data = gfX.vec
        
        # assemble and invert
        Mstar.Assemble()
        A.Assemble()
        inv.Update()
        
        # update position
        r.data = A.mat*gfX.vec
        gfX.vec.data -= tau*inv*r
        
        # update displacement
        gfset.vec.data += gfX.vec-gfXold.vec
        
        if i % 10 == 0:
            scene.Redraw()
        t += tau
        i += 1

Next we consider a dumbbell initial surface.  After running the cell below, rerun the cell above to see the evolution.

In [ ]:
Tend = 0.6
tau = 1e-3

# Reset initial data
# X = dumbell, displacement = dumbell-sphere
gfXold.Set( CF( (x,y,z) ) , definedon=mesh.Boundaries(".*"))
gfX.Set( CF( (5*x, y/2*(1.5+10*sqrt(x**2)**1.7), z/2*(1.5+10*sqrt(x**2)**1.7)) ), definedon=mesh.Boundaries(".*"))
gfset.vec[:] = gfX.vec[:] - gfXold.vec[:]

# auxiliary vector for time stepping scheme
r = gfset.vec.CreateVector()

scene = Draw(Norm(gfset), mesh, deformation=gfset)